In [108]:
import pandas as pd

In [109]:
df = pd.read_csv("cse_class_schedule.csv")

In [110]:
df = df[0:36]

In [111]:
len(df)

36

In [112]:
df["Dates"].unique()

array(['01/22/2025 - 05/06/2025'], dtype=object)

In [113]:
df.columns

Index(['Course', 'Section', 'Title', 'Instructor', 'Dates', 'Days', 'Times',
       'Room', 'Instruction Mode', 'Enrolled'],
      dtype='object')

In [114]:
df["Days"].unique()

array(['TR', 'MWF', 'R', 'MW'], dtype=object)

In [115]:
len(df["Course"].unique())

23

In [116]:
df["Section"]

0      AYY
1      ETH
2     HIRS
3     XION
4        A
5        A
6        B
7        A
8        A
9        B
10    JOS2
11    JOSE
12       A
13       A
14       A
15       A
16       A
17       A
18       B
19       C
20     000
21       A
22       A
23       A
24       A
25       B
26       A
27       B
28       C
29       A
30     000
31       A
32       C
33       D
34       A
35       A
Name: Section, dtype: object

In [117]:
df["Times"]

0      12:30PM-1:50PM
1       1:00PM-1:50PM
2       4:00PM-4:50PM
3       1:00PM-3:50PM
4       5:00PM-7:50PM
5      9:30AM-10:50AM
6       2:00PM-3:20PM
7       5:00PM-6:20PM
8      12:30PM-1:50PM
9       2:00PM-3:20PM
10    10:00AM-11:20AM
11    10:00AM-11:20AM
12      2:00PM-3:20PM
13      2:00PM-3:20PM
14    11:00AM-11:50AM
15     12:30PM-1:50PM
16     12:30PM-1:50PM
17     9:30AM-10:50AM
18    11:00AM-11:50AM
19    10:00AM-10:50AM
20      3:30PM-4:50PM
21     9:30AM-10:50AM
22    11:00AM-11:50AM
23      2:00PM-3:20PM
24     9:30AM-10:50AM
25      5:00PM-6:20PM
26     12:30PM-1:50PM
27      3:30PM-4:50PM
28      5:00PM-6:20PM
29    11:00AM-12:20PM
30    11:00AM-11:50AM
31    12:00PM-12:50PM
32      2:00PM-2:50PM
33      3:00PM-3:50PM
34      5:00PM-6:20PM
35      2:00PM-2:50PM
Name: Times, dtype: object

In [ ]:
"""Create start and end time columns."""
df[['Start Time', 'End Time']] = df['Times'].str.split('-', expand=True)
df['Start Time'] = pd.to_datetime(df['Start Time'], format="%I:%M%p")
df['End Time'] = pd.to_datetime(df['End Time'], format="%I:%M%p")

In [124]:
df["Start Time"] = df["Start Time"].dt.strftime('%H:%M:%S')
df["End Time"] = df["End Time"].dt.strftime('%H:%M:%S')


In [ ]:
def get_schedule(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    # Constraints
    -  Course Days
    -  Course Timing
    -  Course Section has to be unique

    # Algorithm
    -  Select the course with the earliest end time.
    -  Remove all other courses with
        course day == day of selected &
        start times < finish time 
        of the selected course.
    -  Remove all courses with Course == selected course.
    """
    schedule = []

    dataframe = dataframe.sort_values(by='End Time')

    while len(dataframe) > 0:
        earliest_course = dataframe.iloc[0]
        schedule.append(earliest_course)
        dataframe = (
            dataframe[
                dataframe["Start Time"]> earliest_course["End Time"]
            ]
        )
        dataframe = (
            dataframe[
                dataframe["Course"] != earliest_course["Course"]
            ]
        )

    return pd.DataFrame(schedule)


In [210]:
# Unique days: ['TR', 'MWF', 'R', 'MW']
TR_R = df[df['Days'].isin(['TR', 'R'])]

TR_R_schedule = get_schedule(TR_R)

TR_R_schedule


,Course,Section,Title,Instructor,Dates,Days,Times,Room,Instruction Mode,Enrolled,Start Time,End Time
24,CSE 573,A,Introduction to Computer Vision and Image Proc...,Junsong Yuan,01/22/2025 - 05/06/2025,TR,9:30AM-10:50AM,Clemen 322,In person,16/53,09:30:00,10:50:00
29,CSE 578,A,Computational Investment,Zhen Liu,01/22/2025 - 05/06/2025,TR,11:00AM-12:20PM,Cooke 127A,In person,18/20,11:00:00,12:20:00
0,CSE 510,AYY,Special Topic: Algorithms for Mobile Computing...,Sai Roshan Ayyalasomayajula,01/22/2025 - 05/06/2025,TR,12:30PM-1:50PM,Frnczk 408,In person,2/33,12:30:00,13:50:00
13,CSE 546,A,Reinforcement Learning,Alina Vereshchaka,01/22/2025 - 05/06/2025,TR,2:00PM-3:20PM,Knox 104,In person,130/205,14:00:00,15:20:00
20,CSE 562,000,Database Systems,Adekunle Afolabi,01/22/2025 - 05/06/2025,TR,3:30PM-4:50PM,Cooke 121,In person,38/103,15:30:00,16:50:00
7,CSE 526,A,Blockchain Application Development,Bina Ramamurthy,01/22/2025 - 05/06/2025,TR,5:00PM-6:20PM,Hoch 114,In person,10/70,17:00:00,18:20:00


In [211]:
MW_F = df[df['Days'].isin(['MWF', 'MW'])]
MW_F_schedule = get_schedule(MW_F)

MW_F_schedule

,Course,Section,Title,Instructor,Dates,Days,Times,Room,Instruction Mode,Enrolled,Start Time,End Time
19,CSE 560,C,Data Models and Query Languages,Shamsad Parvin,01/22/2025 - 05/06/2025,MWF,10:00AM-10:50AM,Knox 104,In person,167/180,10:00:00,10:50:00
14,CSE 547,A,High Performance Computing 1,Matthew Knepley,01/22/2025 - 05/06/2025,MWF,11:00AM-11:50AM,Norton 210,In person,4/4,11:00:00,11:50:00
31,CSE 587,A,Data Intensive Computing,Shamsad Parvin,01/22/2025 - 05/06/2025,MWF,12:00PM-12:50PM,Knox 104,In person,69/170,12:00:00,12:50:00
1,CSE 510,ETH,Special Topic: Interactive Programming Environ...,Ethan Blanton,01/22/2025 - 05/06/2025,MWF,1:00PM-1:50PM,Norton 209,In person,10/12,13:00:00,13:50:00
35,CSE 590,A,Computer Architecture,Ramalingam Sridhar,01/22/2025 - 05/06/2025,MWF,2:00PM-2:50PM,Knox 110,In person,132/141,14:00:00,14:50:00


In [212]:
combined_schedule = pd.concat([TR_R_schedule, MW_F_schedule])

In [215]:
len(combined_schedule)

11

In [ ]:
combined_schedule.to_csv("generated_schedule.csv")